In [1]:
import pandas as pd
import statistics as st

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
FIVE, FOUR, THREE, TWO, ONE, ZERO = 5, 4, 3, 2, 1, 0

data_table = pd.DataFrame(columns=["league_id", "week", "hour", "minute", "HT", "AT", "HCS", "ACS"])

In [4]:
data_table

,league_id,week,hour,minute,HT,AT,HCS,ACS


In [8]:

def txt_reader(_path):
    file = open(_path, 'r')  # Open the file in read mode
    contents = file.read()  # Read the contents of the file
    file.close()  # Close the file

    return contents

def table_creator(contents):
    
    # data_table = pd.DataFrame(columns=["league_id", "week", "hour", "minute", "HT", "AT", "HCS", "ACS"])

    record_list = []
    weeks = contents.split("WEEK")
    pos = weeks[0].index("League")
    league_no = int(weeks[0][(pos + 7): (pos + 11)])
    for val in weeks[1:]:
        scores = val.split("\n")[1:-1]
        for score in scores:
            temp_dict = {
                "league_id": league_no,
                "week": int((val[:3]).strip()),
                "hour": int(val.split("\n")[0][-8:-6]),
                "minute": int(val.split("\n")[0][-5:-3]),

                "HT": score[0:3],
                "AT": score[8:11],
                "HCS": int(score[4]),
                "ACS": int(score[6])
            }

            # data._append(temp_dict, ignore_index=True)
            record_list.append(temp_dict)
    data = pd.DataFrame(record_list)

    return data

In [14]:
record = txt_reader("league_data/L_6095.txt")
df = table_creator(record)
df

,league_id,week,hour,minute,HT,AT,HCS,ACS
0,6095,1,12,52,EVE,MNC,1,2
1,6095,1,12,52,BRN,TOT,0,1
2,6095,1,12,52,BRI,FUL,1,0
3,6095,1,12,52,LEI,WHU,2,1
4,6095,1,12,52,WOL,ARS,0,0
...,...,...,...,...,...,...,...,...
375,6095,38,14,6,LEI,ARS,2,4
376,6095,38,14,6,MNU,BRN,1,0
377,6095,38,14,6,SOU,WHU,1,2
378,6095,38,14,6,BRI,TOT,3,1


In [18]:
path = "league_data"
records = ["L_6095", "L_6097", "L_6099"]
df_temp = []
for record in records:
    text = txt_reader(f"league_data/{record}.txt")
    df_temp.append(table_creator(text))

multiple_record = pd.concat(df_temp, axis=0)
multiple_record

,league_id,week,hour,minute,HT,AT,HCS,ACS
0,6095,1,12,52,EVE,MNC,1,2
1,6095,1,12,52,BRN,TOT,0,1
2,6095,1,12,52,BRI,FUL,1,0
3,6095,1,12,52,LEI,WHU,2,1
4,6095,1,12,52,WOL,ARS,0,0
...,...,...,...,...,...,...,...,...
375,6099,38,19,10,CRY,FUL,2,2
376,6099,38,19,10,WHU,EVE,0,0
377,6099,38,19,10,ASV,LEI,1,3
378,6099,38,19,10,MNC,CHE,1,2


In [31]:
teams = ['FOR', 'MNC', 'ASV', 'TOT', 'EVE', 'CHE', 'BRN', 'WHU', 'ARS', 'FUL', 
         'NWC', 'BOU', 'LEI', 'LIV', 'WOL', 'MNU', 'LEE', 'SOU', 'BRI', 'CRY']

class Team:
    def __init__(self, name):
        self.name = name
        self.goals_for = 0
        self.goals_against = 0
        self.goal_difference = 0
        self.points = 0
        self.position = 0

team_objects = [Team(team) for team in teams]

def update_table(team_1, team_1_score, team_2, team_2_score):
    team1 = next(team for team in team_objects if team.name == team_1)
    team2 = next(team for team in team_objects if team.name == team_2)

    team1.goals_for = team_1_score
    team1.goals_against = team_2_score
    team1.goal_difference = team_1_score - team_2_score
    team1.points = 3 if team_1_score > team_2_score else 1 if team_1_score == team_2_score else 0

    team2.goals_for = team_2_score
    team2.goals_against = team_1_score
    team2.goal_difference = team_2_score - team_1_score
    team2.points = 3 if team_2_score > team_1_score else 1 if team_2_score == team_1_score else 0

    table = sorted(team_objects, key=lambda x: (-x.points, -x.goal_difference, -x.goals_for, x.name))

    for i, team in enumerate(table):
        team.position = i + 1

    return table

In [33]:
# Example usage
table = update_table('BOU', 2, 'ASV', 1)

# Print the table
for team in table:
    print("Team:", team.name)
    print("Position:", team.position)
    print("Points:", team.points)
    print("Goal Difference:", team.goal_difference)
    print("Goals For:", team.goals_for)
    print("Goals Against:", team.goals_against)
    print()



Team: MNC
Position: 1
Points: 3
Goal Difference: 2
Goals For: 2
Goals Against: 0

Team: BOU
Position: 2
Points: 3
Goal Difference: 1
Goals For: 2
Goals Against: 1

Team: BRI
Position: 3
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: BRN
Position: 4
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: CHE
Position: 5
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: CRY
Position: 6
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: EVE
Position: 7
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: FOR
Position: 8
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: FUL
Position: 9
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: LEE
Position: 10
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: LEI
Position: 11
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: LIV
Position: 12
Points: 0
Goal Difference: 0
Goals For: 0
Goals Against: 0

Team: MNU
Pos

In [19]:
league_record = pd.read_csv("league_record.csv")
league_record

,league_id,week,hour,minutes,HT,AT,HCS,ACS,HL1S,AL1S,HL2S,AL2S,HL3S,AL3S,hl1_stat,al1_stat,hl2_stat,al2_stat,hl3_stat,al3_stat,h_wins,a_wins,h_draws,a_draws,h_loss,a_loss,h_gf,a_gf,h_ga,a_ga,h_gd,a_gd,hs_avg,as_avg,hw_rat,aw_rat,hl_rat,al_rat,hd_rat,ad_rat,ht_points,at_points,ht_pos,at_pos
0,6095,1,12,52,EVE,MNC,1,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,2,2,1,-1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,3,9,1
1,6095,1,12,52,BRN,TOT,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,1,0,-1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,3,7,2
2,6095,1,12,52,BRI,FUL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,1,1,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3,0,1,12
3,6095,1,12,52,LEI,WHU,2,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2,1,1,2,1,-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3,0,2,19
4,6095,1,12,52,WOL,ARS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,1,6,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9875,6230,38,17,6,LIV,NWC,5,0,1,2,3,3,1,0,-1,1,1,1,1,0,24,13,5,11,9,14,81,47,38,54,43,-7,1.666667,1.666667,0.621622,0.351351,0.243243,0.351351,0.135135,0.297297,77,50,2,11
9876,6230,38,17,6,WHU,TOT,1,3,0,2,2,1,2,1,-1,1,1,-1,1,-1,12,19,6,10,20,9,47,62,57,56,-10,6,1.333333,1.333333,0.324324,0.486486,0.513514,0.243243,0.162162,0.270270,42,67,14,4
9877,6230,38,17,6,ASV,MNC,1,3,0,2,1,1,1,3,-1,-1,-1,1,1,1,15,28,11,6,12,4,49,88,39,32,10,56,0.666667,2.000000,0.405405,0.729730,0.297297,0.108108,0.297297,0.162162,56,90,7,1
9878,6230,38,17,6,FOR,EVE,0,1,1,0,1,2,1,0,-1,-1,-1,0,0,-1,7,16,12,10,19,12,32,44,56,41,-24,3,1.000000,0.666667,0.189189,0.405405,0.486486,0.324324,0.324324,0.270270,33,58,19,7


In [21]:
league_record.columns.tolist()

['league_id',
 'week',
 'hour',
 'minutes',
 'HT',
 'AT',
 'HCS',
 'ACS',
 'HL1S',
 'AL1S',
 'HL2S',
 'AL2S',
 'HL3S',
 'AL3S',
 'hl1_stat',
 'al1_stat',
 'hl2_stat',
 'al2_stat',
 'hl3_stat',
 'al3_stat',
 'h_wins',
 'a_wins',
 'h_draws',
 'a_draws',
 'h_loss',
 'a_loss',
 'h_gf',
 'a_gf',
 'h_ga',
 'a_ga',
 'h_gd',
 'a_gd',
 'hs_avg',
 'as_avg',
 'hw_rat',
 'aw_rat',
 'hl_rat',
 'al_rat',
 'hd_rat',
 'ad_rat',
 'ht_points',
 'at_points',
 'ht_pos',
 'at_pos']